In model 2, the model becomes multiple lanes and drivers begins to swap lane.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(8)

# initialize the road, -1 for empty , 0 represent cars with 0 velocity
def initialize_road(length, num_cars):
    lanes = [np.full(length, -1), np.full(length, -1)]
    for lane in lanes:
        car_positions = np.random.choice(length, num_cars//2, replace=False)
        lane[car_positions] = 0
    return lanes

# get the gap 
def get_gap_ahead_same_lane(lane, pos):
    length = len(lane)
    
    # check if the current position is valid
    if pos < 0 or pos >= length:
        raise ValueError("Current position is out of range")
    
    # check if there is a car at the current position
    if lane[pos] == -1:
        raise ValueError("There is no car at the current position")
    
    gap = 0
    for i in range(1, length):
        next_pos = (pos + i) % length
        if lane[next_pos] >= 0:
            return gap
        gap += 1
    return gap

# get forward and backward gaps for other lane
def get_gaps_other_lane(lanes, lane_idx, pos):
    other_lane = lanes[1 - lane_idx]
    length = len(other_lane)
    
    # Forward
    forward_gap = 0
    for i in range(1, length):
        next_pos = (pos + i) % length
        if other_lane[next_pos] >= 0:
            break
        forward_gap += 1
        
    # Backward
    backward_gap = 0    
    for i in range(1, length):
        prev_pos = (pos - i) % length
        if other_lane[prev_pos] >= 0:
            break
        forward_gap += 1
    return forward_gap, backward_gap

# check whether should change the lane
def should_change_lane(lanes, lane_idx, pos, p_change):
    if lanes[lane_idx][pos] == -1:
        return False
        

    gap = get_gap_ahead_same_lane(lanes[lane_idx], pos)
    v = lanes[lane_idx][pos]
    
    # T1: Check if blocked
    if gap >= v + 1:
        return False
        
        
    # T2 & T3: Check if other lane has enough space
    gap_front, gap_back = get_gaps_other_lane(lanes, lane_idx, pos)
    
    if gap_front <= v + 1 or gap_back < 5:
        return False
        
    # T4: Random decision
    if np.random.random() >= p_change:
        return False
    
    return True

# updatge change of lane
def change_lanes(lanes, p_change):
    length = len(lanes[0])
    changes = []

    # record the index and position that would change lane
    for lane_idx in range(2):
        for pos in range(length):
            if should_change_lane(lanes, lane_idx, pos, p_change):
                changes.append((lane_idx, pos))
    
    # change lane
    for lane_idx, pos in changes:
        if lanes[lane_idx][pos] >= 0:
            # get velocity
            v = lanes[lane_idx][pos]
            # change lane
            lanes[lane_idx][pos] = -1
            lanes[1-lane_idx][pos] = v
    return lanes

# update velocities for each step
def update_velocities(lanes, vmax, p):
    for lane_idx in range(2):
        for pos in range(len(lanes[0])):
            if lanes[lane_idx][pos] >= 0:
                v = lanes[lane_idx][pos]
                
                # Acceleration
                if v < vmax:
                    v += 1
                
                # Slowing down
                gap = get_gap_ahead_same_lane(lanes[lane_idx], pos)
                v = min(v, gap)
                
                # Random slowdown
                if v > 0 and np.random.random() < p:
                    v -= 1
                
                lanes[lane_idx][pos] = v
    return lanes

# move cars after get velocity
def move_cars(lanes):
    length = len(lanes[0])
    new_lanes = [np.full(length, -1), np.full(length, -1)]
    
    for lane_idx in range(2):
        for pos in range(length):
            if lanes[lane_idx][pos] >= 0:
                v = lanes[lane_idx][pos]
                new_pos = (pos + v) % length
                new_lanes[lane_idx][new_pos] = v
    
    return new_lanes

# code for update traffic for each step
def update_traffic(lanes, vmax=5, p=0.3, p_change=1):
    lanes = change_lanes(lanes, p_change)
    lanes = update_velocities(lanes, vmax, p)
    lanes = move_cars(lanes)
    return lanes

